In [116]:
from gurobipy import *
import numpy as np
import pandas as pd
from Hourbid import *
from Fibid import *
from blockbid import *


In [117]:
BS_bids = Blockbid()[0]
BD_bids = Blockbid()[1]
HS_bids = HourBid()[0]
HD_bids = HourBid()[1]
FS_bids = Fibid()[0]
FD_bids = Fibid()[1]

In [118]:
L_D = len(BD_bids)
L_S = len(Blockbid()[0])

In [119]:
Delta_S = np.zeros([L_S,24])


Delta_D = np.zeros([L_D,24])
BD_bids  = Blockbid()[1]
for i in range (L_D):
    if BD_bids['Hour'][i]+BD_bids['Duration'][i]<=25:
        Delta_D[i,BD_bids['Hour'][i]-1:(BD_bids['Hour'][i]+BD_bids['Duration'][i])-1] = 1

BS_bids = Blockbid()[0]
for i in range (L_S):
    if BS_bids['Hour'][i]+BS_bids['Duration'][i]<=25:
        Delta_S[i,BS_bids['Hour'][i]-1:(BS_bids['Hour'][i]+BS_bids['Duration'][i])-1] = 1

In [120]:
for i in range(L_D):
    if BD_bids['Hour'][i]==19:
        print(i)

19


In [121]:
BD_bids

,ID,OrderID,Hour,BitType,Quantity,price,Duration,LinkID
0,1110,1,2,B,694.0,599.67,20,NaN
1,1106,1,2,B,562.0,553.92,11,NaN
2,1105,1,15,B,347.0,609.00,4,NaN
3,1102,1,15,B,543.0,581.19,8,NaN
4,1109,1,20,B,808.0,685.34,4,NaN
...,...,...,...,...,...,...,...,...
80,1078,1,2,B,129.0,716.49,19,NaN
81,1199,1,3,B,850.0,444.24,21,NaN
82,1090,1,11,B,172.0,563.73,10,NaN
83,1098,1,6,B,921.0,648.55,19,NaN


In [122]:
Delta_D[0,:]

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 0., 0.])

In [123]:
def find_F(Del,leng,bids):
    Res = [[] for i in range(24)]
    Res_max = [[] for i in range(24)]
    Res_min = [[] for i in range(24)]
    for i in range (24):
        for j in range(leng):
            if Del[j,i] == 1:
                Res[i].append(bids['price'][j])
    for i in range(24):
        Res_max[i] = np.max(Res[i])
        Res_min[i] = np.min(Res[i])
    return(Res_max,Res_min)

FD_min = np.min(FD_bids['price'])
FD_max = np.max(FD_bids['price'])
FS_min = np.min(FS_bids['price'])
FS_max = np.max(FS_bids['price'])

In [124]:
DMax_Res = [[] for i in range(24)]
Dm_Res = [[] for i in range(24)]
SMax_Res = [[] for i in range(24)]
Sm_Res = [[] for i in range(24)]

for i in range(24):
    DMax_Res[i].append(HD_bids['price'][HIndex_D[i][1]])
    DMax_Res[i].append(find_F(Delta_D,L_D,BD_bids)[0][i])
    DMax_Res[i].append(FD_max)
    
for i in range(24):
    SMax_Res[i].append(HS_bids['price'][HIndex_S[i][1]])
    SMax_Res[i].append(find_F(Delta_S,L_S,BS_bids)[0][i])
    SMax_Res[i].append(FS_max)
    
for i in range(24):
    Dm_Res[i].append(HD_bids['price'][HIndex_D[i][0]])
    Dm_Res[i].append(find_F(Delta_D,L_D,BD_bids)[1][i])
    Dm_Res[i].append(FD_min)
    
for i in range(24):
    Sm_Res[i].append(HS_bids['price'][HIndex_S[i][0]])
    Sm_Res[i].append(find_F(Delta_S,L_S,BS_bids)[1][i])
    Sm_Res[i].append(FS_min)

In [125]:
def findmin(num1,num2,num3):
    
    return(min([num1,num2,num3]))

def findmax(num1,num2,num3):

    return(max([num1,num2,num3]))


def put_into_list(maximums,minimums):
    F_min = []
    F_max = []
    for x in range(24):
        F_min.append(findmin(minimums[x][0],minimums[x][1],minimums[x][2]))
        F_max.append(findmax(maximums[x][0],maximums[x][1],maximums[x][2]))

    return(F_min,F_max)

In [126]:
E = Model("Energy")
E.params.NonConvex =2
E.update()
J = np.arange(1,find_largest(HIndex_D)+1)
K = np.arange(1,find_largest(HIndex_S)+1)
H = np.arange(1,25)

Set parameter NonConvex to value 2


In [127]:
xD = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xD')
xS = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xS')
w_D = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS,name ='W_D')
w_S = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS,name ='W_S')
yD = E.addVars(L_D,ub=1,lb=0,  vtype=GRB.BINARY,name ='yD')
yS = E.addVars(L_S,ub=1,lb=0,  vtype=GRB.BINARY,name ='yS')
F_zS = E.addVars(H,lb = 0,vtype=GRB.CONTINUOUS, name='F_ZS')
F_zD = E.addVars(H,lb = 0,vtype=GRB.CONTINUOUS, name='F_ZD')

In [128]:
Data_H_D = HourBid()[1]
Data_H_SP = HourBid()[0]

In [129]:
E.setObjective(sum(0.5*(2*Pair(j,h,Data_H_D,'DP')[0]+xD[h,j]*(Pair(j,h,Data_H_D,'DP')[1]-Pair(j,h,Data_H_D,'DP')[0]))*xD[h,j]*(Pair(j,h,Data_H_D,'DQ')[1]-Pair(j,h,Data_H_D,'DQ')[0])for j in J for h in H)
                -sum(0.5*(2*Pair(j,h,Data_H_SP,'SP')[0]+xS[h,j]*(Pair(j,h,Data_H_SP,'SP')[1]-Pair(j,h,Data_H_SP,'SP')[0]))*xS[h,j]*(Pair(j,h,Data_H_SP,'SQ')[1]-Pair(j,h,Data_H_SP,'SQ')[0])for j in J for h in H)
                +sum(BD_bids['Duration'][i]*BD_bids['Quantity'][i]*BD_bids['price'][i]*yD[i] for i in range(L_D)) - sum(BS_bids['Duration'][i]*BS_bids['Quantity'][i]*BS_bids['price'][i]*yS[i] for i in range(L_S))
,GRB.MAXIMIZE)

In [130]:
gamma = 100000000000000000

        # (2)
E.addConstrs(-sum((Pair(j,h,Data_H_D,'DQ')[1]-Pair(j,h,Data_H_D,'DQ')[0]) * xD[h,j] for j in J ) +sum((Pair(j,h,Data_H_SP,'SQ')[1]-Pair(j,h,Data_H_SP,'SQ')[0])*xS[h,j] for j in J) -Pair(1,h,Data_H_D,'DQ')[0] +Pair(1,h,Data_H_SP,'SQ')[0] 
                    +sum(Delta_S[i,h-1]*BS_bids['Quantity'][i]*yS[i] for i in range(L_S))  -sum(Delta_D[i,h-1]*BD_bids['Quantity'][i]*yD[i] for i in range(L_D))
                    ==0 for h in H) 
 
        
#  for h in H:       # (6)
        # F_z[h]= Sm_Res[h-1][0]+sum((Pair(j,h,Data_H_SP,'SP')[1]-Pair(j,h,Data_H_SP,'SP')[0]) * xS[h,j] for j in J)
# for h in H:        
#         E.addConstr(F_z[h] == sum((Pair(j,h,Data_H_SP,'SP')[1]-Pair(j,h,Data_H_SP,'SP')[0]) * xS[h,j] for j in J))        
#         E.addConstr(F_z[h] == 2000+sum((Pair(j,h,Data_H_D,'DP')[1]-Pair(j,h,Data_H_D,'DP')[0]) * xD[h,j] for j in J))

E.addConstrs(F_zS[h] == Sm_Res[h-1][0]+sum((Pair(j,h,Data_H_SP,'SP')[1]-Pair(j,h,Data_H_SP,'SP')[0]) * xS[h,j] for j in J)for h in H)        
E.addConstrs(F_zD[h] == DMax_Res[h-1][0]+sum((Pair(j,h,Data_H_D,'DP')[1]-Pair(j,h,Data_H_D,'DP')[0]) * xD[h,j] for j in J) for h in H)
# (8)

E.addConstrs(-BS_bids['Duration'][j]* BS_bids['price'][j]+ sum(Delta_S[j,h-1]*F_zS[h] for h in H ) <= yS[j]*gamma for j in range(L_S))
# (9)

E.addConstrs(BD_bids['Duration'][j]*BD_bids['price'][j]- sum(Delta_S[j,h-1]*F_zD[h] for h in H) <= yD[j]*gamma for j in range(L_D))
        
        # E.addConstr(if Blockbid['LinkID'][j] == 'NaN': -Blockbid['Duration'][j]* Blockbid['price'][j]+ sum(Delta_S[j,h]*F_z[h] for j in J) <= yd[j]*gamma)
        # E.addConstr(if Blockbid['LinkID'][j] == 'NaN': Blockbid['Duration'][j]*Blockbid['price'][j]+ sum(Delta_S[j,h]*F_z[h] for j in J) <= yd[j]*gamma)
        
# (3)(4)
# Demand
E.addConstrs(w_D[h,1]<=xD[h,1] for h in H)
E.addConstrs(xD[h,1] <=1 for h in H)
E.addConstrs(w_D[h,j]<=xD[h,j] for j in range(2,find_largest(HIndex_D)+1) for h in H)
E.addConstrs(xD[h,j]<=w_D[h,j-1] for j in range(2,find_largest(HIndex_D)+1) for h in H)
# Supply
E.addConstrs(w_S[h,1]<=xS[h,1] for h in H)
E.addConstrs(xS[h,1] <=1 for h in H)
E.addConstrs(w_S[h,j]<=xS[h,j] for j in range(2,find_largest(HIndex_S)+1) for h in H)
E.addConstrs(xS[h,j]<=w_S[h,j-1] for j in range(2,find_largest(HIndex_S)+1) for h in H)



{(2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 8): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11): <gurobi.Constr *Awaiting Model Update*>,
 (2, 12): <gurobi.Constr *Awaiting Model Update*>,
 (2, 13): <gurobi.Constr *Awaiting Model Update*>,
 (2, 14): <gurobi.Constr *Awaiting Model Update*>,
 (2, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 16): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17): <gurobi.Constr *Awaiting Model Update*>,
 (2, 18): <gurobi.Constr *Awaiting Model Update*>,
 (2, 19): <gurobi.Constr *Awaiting Model Update*>,
 (2, 20): <gurobi.Constr *Awaiting Model

In [131]:
E.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 96176 rows, 96152 columns and 293053 nonzeros
Model fingerprint: 0xa5b84b95
Model has 47952 quadratic objective terms
Variable types: 95952 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+17]
  Objective range  [1e+02, 1e+07]
  QObjective range [2e-02, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 48100 rows and 47973 columns
Presolve time: 0.19s
Presolved: 48076 rows, 48179 columns, 188872 nonzeros
Presolved model has 47952 quadratic objective terms
Variable types: 47980 continuous, 199 integer (199 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   36941   -5.9230876e+08   4.008724e+04   0.

In [135]:
E.getAttr('x',yD)
E.getAttr('x',F_zD)
E.getAttr('x',xS)


{(1, 1): 1.0,
 (1, 2): 1.0,
 (1, 3): 1.0,
 (1, 4): 1.0,
 (1, 5): 1.0,
 (1, 6): 1.0,
 (1, 7): 1.0,
 (1, 8): 1.0,
 (1, 9): 1.0,
 (1, 10): 1.0,
 (1, 11): 1.0,
 (1, 12): 1.0,
 (1, 13): 1.0,
 (1, 14): 1.0,
 (1, 15): 1.0,
 (1, 16): 1.0,
 (1, 17): 1.0,
 (1, 18): 1.0,
 (1, 19): 1.0,
 (1, 20): 1.0,
 (1, 21): 1.0,
 (1, 22): 1.0,
 (1, 23): 1.0,
 (1, 24): 1.0,
 (1, 25): 1.0,
 (1, 26): 1.0,
 (1, 27): 1.0,
 (1, 28): 1.0,
 (1, 29): 1.0,
 (1, 30): 1.0,
 (1, 31): 1.0,
 (1, 32): 1.0,
 (1, 33): 1.0,
 (1, 34): 1.0,
 (1, 35): 1.0,
 (1, 36): 1.0,
 (1, 37): 1.0,
 (1, 38): 1.0,
 (1, 39): 1.0,
 (1, 40): 1.0,
 (1, 41): 1.0,
 (1, 42): 1.0,
 (1, 43): 1.0,
 (1, 44): 1.0,
 (1, 45): 1.0,
 (1, 46): 1.0,
 (1, 47): 1.0,
 (1, 48): 1.0,
 (1, 49): 1.0,
 (1, 50): 1.0,
 (1, 51): 1.0,
 (1, 52): 1.0,
 (1, 53): 1.0,
 (1, 54): 1.0,
 (1, 55): 1.0,
 (1, 56): 1.0,
 (1, 57): 1.0,
 (1, 58): 1.0,
 (1, 59): 1.0,
 (1, 60): 1.0,
 (1, 61): 1.0,
 (1, 62): 1.0,
 (1, 63): 1.0,
 (1, 64): 1.0,
 (1, 65): 1.0,
 (1, 66): 1.0,
 (1, 67): 1.0,
 (1,

In [ ]:
Res_D = []
Res_S = []
for i in range(1,1000):

    Res_D.append(E.getAttr('x',xD)[(1,i)])
    Res_S.append(E.getAttr('x',xS)[(1,i)])

In [ ]:
for i in range(len(Res_D)):
    if Res_D[i] >= 1e-8 and Res_D[i] <= 0.999:
        print(i+1)
for i in range(len(Res_S)):
    if Res_S[i] >= 1e-8 and Res_S[i] <= 0.999:
        print(i+1)

320
320


In [ ]:
# def delta(SD):
#     if  SD == 'D':
#         BD_bids  = Blockbid()[1]
#         for i in range (L_D):
#             if BD_bids['Hour'][i]+BD_bids['Duration'][i]<=24:
#                 Delta_D[i,BD_bids['Hour'][i]-1:BD_bids['Hour'][i]+BD_bids['Duration'][i]] = 1
#         return(Delta_D)
#     if  SD == 'S':
#         BS_bids = Blockbid()[0]
#         for i in range (L_S):
#             if BS_bids['Hour'][i]+BS_bids['Duration'][i]<=24:
#                 Delta_S[i,BS_bids['Hour'][i]-1:BS_bids['Hour'][i]+BS_bids['Duration'][i]] = 1
#         return(Delta_S)


BD_bids  = Blockbid()[1]
for i in range (L_D):
    if BD_bids['Hour'][i]+BD_bids['Duration'][i]<=24:
        Delta_D[i,BD_bids['Hour'][i]-1:BD_bids['Hour'][i]+BD_bids['Duration'][i]] = 1

BS_bids = Blockbid()[0]
for i in range (L_S):
    if BS_bids['Hour'][i]+BS_bids['Duration'][i]<=24:
        Delta_S[i,BS_bids['Hour'][i]-1:BS_bids['Hour'][i]+BS_bids['Duration'][i]] = 1


In [ ]:
len(F_z)

24

In [ ]:
BD_bids  = Blockbid()[1]
BD_bids['Hour'][0]+BD_bids['Duration'][0]

22